In [1]:
import numpy as np
import pandas as pd 

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer


In [3]:
df = pd.read_csv("covid_toy.csv")

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.shape

(100, 6)

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(df.drop(columns=["has_covid"]) , 
                                                       df["has_covid"] , test_size=0.2)

In [12]:
X_train

,age,gender,fever,cough,city
61,81,Female,98.0,Strong,Mumbai
91,38,Male,NaN,Mild,Delhi
47,18,Female,104.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
49,44,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
57,49,Female,99.0,Strong,Bangalore
93,27,Male,100.0,Mild,Kolkata
69,73,Female,103.0,Mild,Delhi
60,24,Female,102.0,Strong,Bangalore


$ Normal \ Method $

In [ ]:
# imputation(handeling missing data)
si = SimpleImputer()
si.fit(X_train[['fever']])
X_train_fever = si.transform(X_train[['fever']])
X_test_fever = si.transform(X_test[["fever"]])

In [15]:
X_train_fever.shape

(80, 1)

In [16]:
# ordinal encoding
oe = OrdinalEncoder(categories=[["Mild" , "Strong"]])
oe.fit(X_train[["cough"]])  
X_train_cough = oe.transform(X_train[["cough"]])  
X_test_cough = oe.transform(X_test[["cough"]])  

In [17]:
X_train_cough.shape

(80, 1)

In [20]:
# Onehot encoding 
ohe = OneHotEncoder(drop="first" , dtype=np.int32)
X_train_gender_city = ohe.fit_transform(X_train[["gender" ,"city"]]).toarray()
X_test_gender_city = ohe.transform(X_test[["gender" ,"city"]]).toarray()

In [22]:
X_train_gender_city[:5] , X_test_gender_city[:5]

(array([[0, 0, 0, 1],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 1],
        [1, 0, 0, 1]], dtype=int32),
 array([[0, 0, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 1, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0]], dtype=int32))

In [23]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [27]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

$ColumnTransformer$

In [28]:
from sklearn.compose import ColumnTransformer

In [30]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [33]:
transformer.fit_transform(X_train)

array([[ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [100.80555556,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  38.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  42.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  44.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  60.        ],
       [102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [100.80555556,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  34.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.    

In [34]:
transformer.transform(X_test)

array([[103.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  16.        ],
       [100.80555556,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  20.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  47.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  64.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [101.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.    